### Sentiment analysis with an attention transformer
* sentiment bit adapted from: https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
* transformer from: bloem

In [2]:
import torch
import torchtext
import torch.nn as nn
import torch.nn.functional as F
from torchtext.datasets import text_classification
from torch.utils.data import DataLoader

import re
from torchtext.data.utils import ngrams_iterator
from torchtext.data.utils import get_tokenizer

from torch.utils.data.dataset import random_split

import time
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Data

In [5]:
BATCH_SIZE = 16
NGRAMS = 2

if not os.path.isdir('../data/NLP/text_classification'):
    os.mkdir('../data/NLP/text_classification')
    
train_dataset, test_dataset = text_classification.DATASETS['AG_NEWS'](
    root='../data/NLP/text_classification', ngrams=NGRAMS, vocab=None)

ag_news_csv.tar.gz: 11.8MB [00:00, 15.9MB/s]
120000lines [00:09, 13085.34lines/s]
120000lines [00:17, 7040.62lines/s]
7600lines [00:01, 7261.86lines/s]


### Model definitions

In [6]:
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()
        
    def init_weights(self):
        init_range = 0.5
        self.embedding.weight.data.uniform_(-init_range, init_range)
        self.fc.weight.data.uniform_(-init_range, init_range)
        self.fc.bias.data.zero_()
    
    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [11]:
class AttentionLayer(nn.Module):
    def __init__(self, k, num_heads=8):
        super(AttentionLayer, self).__init__()
        self.k = k
        self.num_heads = num_heads
        
        # determine queries, keys, values
        self.key_layer = nn.Linear(self.k, self.k * self.num_heads, bias=False)
        self.query_layer = nn.Linear(self.k, self.k * self.num_heads, bias=False)
        self.value_layer = nn.Linear(self.k, self.k * self.num_heads, bias=False)
        
        # project down all cat-ed heads
        self.unify_layer = nn.Linear(heads * k, k)
    
    def forward(self, x):
        # get batch size, t sentences of k items
        b_sz, t_sz, k_sz = x.size()
        h_sz = self.num_heads
        
        keys = self.key_layer(x).view(b_sz, t_sz, h_sz, self.k)
        queries = self.query_layer(x).view(b_sz, t_sz, h_sz, self.k)
        values = self.value_layer(x).view(b_sz, t_sz, h_sz, self.k)
    
        # compute dot products (k x k). Same op for every head, so fold in to the
        # batch dim
        # q, k, v, (b, t, h, k) -> (b, h, t, k) -> (bh, t, k)
        # and for the key (bh, t, k) -> (bh, k, t) to be able to use bmm
        #
        keys = keys.transpose(1, 2).continuous().view(b_sz * h_sz, t_sz, k_sz)
        queries = queries.transpose(1, 2).continuous().view(b_sz * h_sz, t_sz, k_sz)
        values = values.transpose(1, 2).continuous().view(b_sz * h_sz, t_sz, k_sz)
        
        # intermediate scaling
        queries = queries / ( k ** (1./4.))
        keys = keys / ( k ** (1./4.))
        
        # final transpose for the bmm, out -> (b*h, t, t)
        raw_weights = torch.bmm(queries, keys.transpose(1, 2))
        
        # row wise softmax normalize
        weights = F.softmax(raw_weights, dim=2)
        
        # apply self attention to the values
        out = torch.bmm(weights, values).view(b_sz, h_sz, t_sz, k_sz)
        
        # Unify attention heads
        # reshuffle (b, h, t, k) -> (b, t, h, k) -> (b, t, h*k) with all the heads catted
        # ontop of each other to be able to down project
        out = out.transpose(1, 2).continuous().view(b_sz, t_sz, h_sz * k_sz)
        
        # project down
        out = self.unify_layer(out)
        
        return out
    
    
class TransformerBlock(nn.Module):
    def __init__(self, k, num_heads):
        super(TransformerBlock, self).__init__()
        
        self.attention = AttentionLayer(k, num_heads)
        
        self.layer_norm1 = nn.LayerNorm(k)
        self.layer_norm2 = nn.LayerNorm(k)
        
        self.mlp = nn.Sequential(
            nn.Linear(k, 4 * k),
            nn.Relu(),
            nn.Linear(4 * k, k)
        )
    
    def forward(self, x):
        # Attention block
        x_att = self.attention(x)
        # Residual + norm
        x = self.layer_norm1(x + x_att)
        # MLP
        x_mlp = self.mlp(x)
        out = self.layer_norm2(x + x_mlp)
        return out
    
    
class Transformer(nn.Module):
    def __init__(self, k, num_heads, depth, seq_length, num_tokens, num_classes):
        super(Transformer, self).__init__()
        
        self.num_tokens = num_tokens
        
        # Embedding tokens and position layers
        self.token_embed_layer = nn.Embedding(num_tokens, k)
        self.position_embed_layer = nn.Embedding(seq_length, k)
        
        # Transformer blocks
        self.tf_network = []
        for _ in range(depth):
            tf_network.append(TransformerBlock(k, num_heads))

        self.tf_network = nn.Sequential(*self.network)
        
        # Sequence to class output
        self.output_layer = nn.Linear(k, num_classes)
        
    def forward(self, x):
        # in (b, t) tensor with int values representing words
        # out (b, c) tensor logprobs over c classes
        
        # generate token embeddings
        tokens = self.token_embed_layer(x)
        
        b_sz, t_sz, k_sz = tokens.size()
        
        # generate position embeddings
        positions = torch.arange(t_sz)
        positions = self.position_embed_layer(positions)[None, :, :].expand(b_sz, t_sz, k_sz)
        
        x = tokens + positions
        
        # Transformer forward
        x = self.tf_network(x)
        
        # Average pool over t dimension and project to class probabilities
        x = self.output_layer(x.mean(dim=1))
        
        # Optional (auto-regressive) transformer
        # no looking ahead, enforce via mask, prior to softmax
#         indices = torch.triu_indices(t, t, offset=1)
#         x[:, indices[0], indices[1]] = float('-inf')
        
        
        out = F.log_softmax(x, dim=1)
        
        return out  

### Settings

In [7]:
VOCAB_SIZE = len(train_dataset.get_vocab())
EMBED_DIM = 32
NUN_CLASS = len(train_dataset.get_labels())
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS).to(device)

In [ ]:
VOCAB_SIZE = len(train_dataset.get_vocab())
EMBED_DIM = 32
NUN_CLASS = len(train_dataset.get_labels())
# k, num_heads, depth, seq_length, num_tokens, num_
seq_length = 
model = Transformer(EMBED_DIM, 1, 1,   seq_length, VOCAB_SIZE, NUN_CLASS).to(device)

### Batch functions


In [8]:
def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text]
    # final offsets via cumsum, note you don't need an offset for the last entry     
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)
    return text, offsets, label


def train_test(dataset_subset, is_train):
    batch_loss = 0
    batch_acc = 0
    data_loader = DataLoader(dataset_subset, batch_size=BATCH_SIZE, shuffle=is_train,
                     collate_fn=generate_batch)
    
    for i, (text, offsets, labels) in enumerate(data_loader):
        if is_train:
            optimizer.zero_grad()
        text, offsets, labels = text.to(device), offsets.to(device), labels.to(device)
        
        if is_train:
            output = model(text, offsets)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
        else:
            with torch.no_grad():
                output = model(text, offsets)
                loss = criterion(output, labels)
        
        batch_loss += loss.item()
        batch_acc += (output.argmax(1) == labels).sum().item()
    
    if is_train:
        scheduler.step()
    
    return batch_loss / (len(dataset_subset)), batch_acc / (len(dataset_subset))


### Main epoch loop

In [ ]:
N_EPOCHS = 5
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

train_len = int(len(train_dataset) * 0.95)
sub_train_, sub_valid_ = \
    random_split(train_dataset, [train_len, len(train_dataset) - train_len])

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_test(sub_train_, True)
    valid_loss, valid_acc = train_test(sub_valid_, False)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

### Testing

In [9]:
ag_news_label = {1 : "World",
                 2 : "Sports",
                 3 : "Business",
                 4 : "Sci/Tec"}


def predict(text, model, vocab, ngrams):
    tokenizer = get_tokenizer("basic_english")
    with torch.no_grad():
        text = torch.tensor([vocab[token]
                            for token in ngrams_iterator(tokenizer(text), ngrams)])
        
        text = text.to(device)
        output = model(text, torch.tensor([0]).to(device))
        return output.argmax(1).item() + 1

In [ ]:
vocab = train_dataset.get_vocab()

ex1 = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

ex2 = "António Guterrez, the United Nations secretary general, \
contrasted the “leadership” and “mobilisation” shown by the world’s \
youth on the climate emergency with the lack of action by governments, \
which were failing to keep up with the urgency of the problem \
despite increasing signs that the climate was reaching breakdown."


# model = model.to("cpu")

print("This is %s news" %ag_news_label[predict(ex1, model, vocab, 2)])
print("This is %s news" %ag_news_label[predict(ex2, model, vocab, 2)])